In [5]:
# Import BW25 packages
import bw2data as bd
import bw2io as bi
import brightway2 as bw 
import bw2calc as bc

In [6]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Brighway\Libaries')

In [7]:
# Importing self-made libaries
import standards as s
import Monte_Carlo as MC
import life_cycle_assessment as lc
import LCA_plots as lp
import box_plot as bp
import reload_lib as rl


lib = [s, MC, lc, lp, bp]
rl.reload_lib(lib)


In [59]:
all_methods = [m for m in bw.methods if 'ReCiPe 2016 v1.03, midpoint (H)' in str(m) and 'no LT' not in str(m)] # Midpoint

# Obtaining the endpoint categories and ignoring land transformation
endpoint = [m for m in bw.methods if 'ReCiPe 2016 v1.03, endpoint (H)' in str(m) and 'no LT' not in str(m) and 'total' in str(m)]


# Combining midpoint and endpoint, where endpoint is added to the list of the midpoint categories
for meth in endpoint:
    all_methods.append(meth)

# Step 2: Dictionary to store new method names
new_methods = {}
check = {}

method_name_new_mid= all_methods[0][0] + ' Runes edition'
method_name_new_end = all_methods[-1][0] + ' Runes edition'
if 'OG' in method_name_new_end:
    method_name_new_end = method_name_new_end.replace(f'OG ', '')


# Step 3: Process all methods
if method_name_new_mid not in [m[0] for m in list(bd.methods)] or method_name_new_end not in [m[0] for m in list(bd.methods)]:
    print('Does not exist')
    for metod in all_methods:
        recipe_no_bio_CO2 = []  # Temporary storage for filtered CFs
        if metod[1] not in check.keys():
            check[metod[1]] = None
            method = bw.Method(metod)
            cf_data = method.load()
            # Step 4: Filter CFs
            for cf_name, cf_value in cf_data:
                flow_object = bw.get_activity(cf_name)
                flow_name = flow_object['name'] if flow_object else "Unknown Flow"
                if 'non-fossil' not in flow_name:
                    recipe_no_bio_CO2.append((cf_name, cf_value))
                else:
                    recipe_no_bio_CO2.append((cf_name, 0))
            # Step 5: Define a new method key
            new_metod = (metod[0] + ' Runes edition', metod[1], metod[2])
            new_method_key = new_metod
            new_method = bw.Method(new_method_key)
            new_method.register()
            new_method.write(recipe_no_bio_CO2)

            # Step 6: Store the new method
            new_methods[metod] = new_method_key
            print(f"New method created: {new_method_key} with {len(recipe_no_bio_CO2)} CFs")
else:
    print('Method already exist')


            

Method already exist
